# Recomendação de Filmes

Vamos utilizar o surprise para resolver um problema de recomendação de filmes, baseado em notas de avaliação dos usuário. O dataset utlizado foi o [MovieLens Latest Datasets](https://grouplens.org/datasets/movielens/), com 100k avaliações de de 600 usários em 9k filmes.

A recomendação de filmes é algo comum encontrado nas plataformas de streaming, mas a recomendação no geral está presente em muitas outras aplicações.

## 1. Importando as Bibliotecas

Vamos começar instalando a biblioteca surprise.

In [1]:
!pip install surprise

     |████████████████████████████████| 11.8MB 6.0MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670946 sha256=2165f2ee6fab94f9ad1916804cb7e16ca88d59ef882eebd3f0cc05809dc8d1b5
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


Agora podemos importar todos os pacotes q vamos utilizar.

In [2]:
import pandas as pd
import numpy as np

from surprise import Reader, Dataset
from surprise.prediction_algorithms.knns import KNNBaseline
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.matrix_factorization import SVDpp
from surprise.prediction_algorithms.slope_one import SlopeOne
from surprise.prediction_algorithms.co_clustering import CoClustering
from surprise import accuracy


## 2. Carregando Dados

Vamos carregar todas as nossas base de dados.

In [3]:
links = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/links.csv')
links.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [4]:
movies = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/movies.csv')
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [5]:
ratings = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/ratings.csv')
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


In [6]:
tags = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/tags.csv')
tags.head(2)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996


## 3. EDA

Agora vamos dar uma analisada nos dados que temos. Sigamos com a nossa Análise Exploratória dos Dados.

In [7]:
# Juntando as informações de filmes e avaliações
filmes = ratings.join(movies.set_index('movieId'), on='movieId')

In [8]:
print('Tamanho da Base de Filmes: ', filmes.shape)

Tamanho da Base de Filmes:  (100836, 6)


In [9]:
filmes.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [10]:
# Números dos datasets
print('Quantidade de Filmes Avaliados: ',
filmes['movieId'].value_counts().shape[0])

print('Quantidade de Usuários Avaliando: ',
filmes['userId'].value_counts().shape[0])

print('Quantidade de Avaliações: ',
ratings.shape[0])

Quantidade de Filmes Avaliados:  9724
Quantidade de Usuários Avaliando:  610
Quantidade de Avaliações:  100836


In [11]:
# Quantidade de Avaliações TOP5 Filmes
filmes['title'].value_counts().head()

Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
Name: title, dtype: int64

In [12]:
# Quantidade de Avaliações LOW5 Filmes
filmes['title'].value_counts().tail()

Radio Day (2008)              1
Fading Gigolo (2013)          1
Great Ziegfeld, The (1936)    1
Stoned (2005)                 1
Desperate Living (1977)       1
Name: title, dtype: int64

In [13]:
# Quantidade de Avaliações TOP5 Usuários
filmes['userId'].value_counts().head()

414    2698
599    2478
474    2108
448    1864
274    1346
Name: userId, dtype: int64

In [14]:
# Quantidade de Avaliações LOW5 Usuários
filmes['userId'].value_counts().tail()

406    20
595    20
569    20
431    20
442    20
Name: userId, dtype: int64

In [15]:
# Avaliações do usuário 414
filmes.query('userId == 414').head()

,userId,movieId,rating,timestamp,title,genres
62294,414,1,4.0,961438127,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
62295,414,2,3.0,961594981,Jumanji (1995),Adventure|Children|Fantasy
62296,414,3,4.0,961439278,Grumpier Old Men (1995),Comedy|Romance
62297,414,5,2.0,961437647,Father of the Bride Part II (1995),Comedy
62298,414,6,3.0,961515642,Heat (1995),Action|Crime|Thriller


In [16]:
# Avaliações do usuário 414
filmes.query('userId == 414 and movieId == 168248')

,userId,movieId,rating,timestamp,title,genres


## 4. Modelo

In [17]:
# Configuração para treinamento
reader = Reader(rating_scale=(0,5))
# Seleção das variáveis para o modelo
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
# Divisão dos dados de treino e teste
trainset, testset = train_test_split(data, test_size=.2)

In [18]:
# Configurações das medidas de similaridade
sim_options = { 'name': 'pearson_baseline', 'user_based': True }

### 4.1 Treinamento do Modelo


Um algoritmo básico que leva em consideração uma classificação de linha de base.

In [19]:
knn = KNNBaseline(k=33, sim_options=sim_options)
knn.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Um algoritmo simples, mas preciso.

In [20]:
slo = SlopeOne()
slo.fit(trainset)

Um algoritmo baseado em co-clustering. 
Os clusters são atribuídos usando um método de otimização direto, muito parecido com o k-means.


In [22]:
co = CoClustering(n_epochs=10, verbose=True)
co.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


### 4.2 Avaliação do Modelo

Para avaliar os modelos utilizamos o erro quadrático médio das previsões.

In [23]:
predictions_knn = knn.test(testset)
accuracy.rmse(predictions_knn)

RMSE: 0.8773


0.877322331436563

In [25]:
predictions_co = co.test(testset)
accuracy.rmse(predictions_co)

RMSE: 0.9480


0.9480022459676123

In [26]:
predictions_slo = slo.test(testset)
accuracy.rmse(predictions_slo)

RMSE: 0.9029


0.9029067260323603

### 4.3 Predição do Modelo

Vamos ver os parâmetros que são atribuidos as predições.

Parameters:	
* uid – The (raw) user id. See this note.
* iid – The (raw) item id. See this note.
* r_ui (float) – The true rating rui.
* est (float) – The estimated rating r^ui.
* details (dict) – Stores additional details about the prediction that might be useful for later analysis.

In [27]:
# Predição com os dados de teste
predictions_co[:5]

[Prediction(uid=560, iid=80463, r_ui=4.0, est=3.777660050341529, details={'was_impossible': False}),
 Prediction(uid=414, iid=8874, r_ui=4.0, est=3.60964942827203, details={'was_impossible': False}),
 Prediction(uid=222, iid=103984, r_ui=2.0, est=3.6810475549768475, details={'was_impossible': False}),
 Prediction(uid=489, iid=6793, r_ui=0.5, est=0.9735489301387612, details={'was_impossible': False}),
 Prediction(uid=551, iid=318, r_ui=4.0, est=4.625458658245941, details={'was_impossible': False})]

A função trainset utiliza um id interno para serem utilizados no modelo, eles podem ser obtidos através da função to_inner, e a função to_raw retorna os valores originais.

In [39]:
# uid=247, iid=364 Conversão para o id interno
print('Conversão Interna')
print('User:', co.trainset.to_inner_uid(247))
print('Movie:', co.trainset.to_inner_iid(364))

# uid=247, iid=364 Conversão para o id externo
print('Conversão Externa')
print('User:', co.trainset.to_raw_uid( co.trainset.to_inner_uid(247) ))
print('Movie:', co.trainset.to_raw_iid( co.trainset.to_inner_iid(364) ))

Conversão Interna
User: 159
Movie: 9
Conversão Externa
User: 247
Movie: 364


#### 4.3.1. Função Análise de Recomendação

Passando um ID de usuário e um ID de um filme, a função vai retornar a Estimativa de Avaliação do usuário para o filme informado. Caso seja um filme já avaliado pelo usuário retorna também a avaliação real.

In [29]:
def recomenda_filme(userId,movieId):
  # ID do usuário para predição
  uid = userId
  # ID do filme para predição
  iid = movieId 
  nome_filme = movies.query('movieId == @movieId')['title'].values[0]
  print('Filme:', nome_filme)
  print('Usuário:', userId)
  if filmes.query('userId == @userId and movieId == @movieId')['title'].values.size == 0:
    print('Usuário não avaliou o filme!')
  else:
    nota_filme = ratings.query('userId == @userId and movieId == @movieId')['rating'].values[0]
    print('Avaliação do usuário:', nota_filme)
  # Predição baseada no melhor modelo
  print('Estimativa de Avaliação[0-5]:', round(co.predict(co.trainset.to_raw_uid(uid), co.trainset.to_raw_iid(iid))[3], 2))

In [30]:
recomenda_filme(406,3)

Filme: Grumpier Old Men (1995)
Usuário: 406
Usuário não avaliou o filme!
Estimativa de Avaliação[0-5]: 3.9


In [31]:
recomenda_filme(247,364) 

Filme: Lion King, The (1994)
Usuário: 247
Avaliação do usuário: 5.0
Estimativa de Avaliação[0-5]: 3.02


#### 4.3.2. Função TOP Recomendações

Passando um ID de usuário e a quantidade de recomendações, a função informa em ordem de recomendação os filmes para o usuário informado.

In [32]:
def top_n(userId,n):
  # Selecionando apenas os filmes do treinamento
  lista_filmes_treino = []
  for x in trainset.all_items():
    lista_filmes_treino.append(trainset.to_raw_iid(x))
  # Selecionando os filmes do treinamento que o usuário não avaliou
  filmes_user = ratings.query('userId == @userId')['movieId'].values
  filmes_user_nao = movies.query('movieId not in @filmes_user')
  filmes_user_nao = filmes_user_nao.query('movieId in @lista_filmes_treino')['movieId'].values
  # Criando um ranking para o usuário para os filmes não avaliados
  ranking=[]
  for movieId in filmes_user_nao:
    ranking.append((movieId, co.predict(co.trainset.to_inner_uid(userId), co.trainset.to_inner_iid(movieId))[3]))
  # Ordenando os TOP filmes avaliados
  ranking.sort(key=lambda x: x[1], reverse=True)
  # Selecionando os Ids dos filmes
  x,_ = zip(*ranking[:n])
  # Listando os nomes dos filmes em ordem de recomendação
  return movies.query('movieId in @x')['title'].copy().reset_index(drop=True)

In [33]:
top_n(414,5)

0      Beyond Bedlam (1993)
1          Boys Life (1995)
2      House Party 3 (1994)
3              Andre (1994)
4    Denise Calls Up (1995)
Name: title, dtype: object

In [34]:
top_n(406,5)

0                Black Sheep (1996)
1           Immortal Beloved (1994)
2              Beyond Bedlam (1993)
3    Stuart Saves His Family (1995)
4              Shallow Grave (1994)
Name: title, dtype: object